In [ ]:
import proveit
from proveit import Function, Lambda, Operation
from proveit import f, p, x, Y, N, P, Z
from proveit.logic import Implies
%begin _curry_paradox_hoss

### I. Original Content

From Hoss: The original contents of this notebook were originally found in the `_curry_paradox_.ipynb` nb at the top level, but that nb was refusing to be recognized by the system as an executable notebook, so I have copied its contents here and continued to experiment. The original content is still available in the original nb in its original location. I have tried to provide comments throughout to preserve the sense of the original content.

From Hoss: the following Lambda() call and the follow-up prove() call was originally here w/out the try-except block. Perhaps they are old enough that they used to work, or perhaps they only work with temporary changes to the underlying python base code in Prove-It? In any case, I've wrapped the Lambda attempt in a try-except block and commented out the Function().prove() call for now, to preserve the original content:

In [ ]:
try:
    Lambda(x, Operation(x, x))
except Exception as the_exception:
    print(f'Exception: {the_exception}')

In [ ]:
# Function(P, P).prove([Function(P, P)]).instantiate({P:Lambda(x, Operation(x, x))})

In [ ]:
# [P -> P](x -> x)

This comment block originally from Wayne (probably):<br/>
In order to get this to go through in Prove-It, I needed to remove a restriction that disallows the operator of an Operation (or Function, which is simply an Operation with a different default style).  (I also had to disable the "generic version" feature because that has an infinite recursion problem in this case).

Comment from Hoss: I have now commented-out the following cells, because it cannot run without changes to the underlying python system code. The relevant change in the operation.py code is likely at approx line 75 that looks like this and catches the case where the operator of an Operation (a Function is a special case of an Operation) is a Lambda map:
```
if isinstance(self.operator, Lambda):
    raise_bad_operator_type(self.operator)
```

This looks like an attempt to define the Curry Y (fixed-point) combinator. It turns out that even when the lambda-as-operator restriction is relaxed, the definition fails because of a max recursion error, possibly because of the general requirement of immediate and exhaustive beta reduction for application of Lambda terms:

In [ ]:
# Y = Lambda(f, Function(Lambda(x, Function(f, Function(x, x))), Lambda(x, Function(f, Function(x, x)))))

In [ ]:
N = Lambda(p, Implies(p, Z))

Comment originally from Wayne (probably): The following leads to an infinite recursion, not allowing us to move further to prove a contradiction via Curry's paradox (in this manner, at least)

In [ ]:
# X = Y.apply(N)

From Hoss: the $Y$, $N$, and $X$ definitions above appear to reflext the example/discusions in the _Wikipedia_ entry for Curry's paradox (https://en.wikipedia.org/wiki/Curry%27s_paradox), particular in the sections on “Lambda calculus with restricted minimal logic” and “Sentential logic.”

### II. A strategy proposed by Deepak

From Hoss: this entire section, on a strategy proposed by Deepak, was in the original content. I have:</br>
<ul>
    <li>added a try-except block on the g.apply(g) cell (max recursion hit);</li>
    <li>added a few comments here and there as I explored possibilities;</li>
    <li>modified a few cells to change a list given as a positional argument to a 'assumptions=' argument.</li>
</ul>


Attempting Deepak's proposed strategy

In [ ]:
import proveit
from proveit import Lambda, Operation, Function
from proveit import g, P, A, alpha, beta
from proveit.logic import Implies, Equals

In [ ]:
g = Lambda(P, Implies(Function(P, P), A))

In [ ]:
try:
    g.apply(g)
except Exception as the_exception:
    print(f'Exception: {the_exception}')

In [ ]:
# inserted by Hoss, to reset the def of 'g'
from proveit import g

In [ ]:
g_def = Equals(g, Lambda(P, Implies(Function(P, P), A)))

In [ ]:
gg = Function(g, g)

In [ ]:
# From hoss: here I changed the orginal positional arg '[g_def]' to 'assumptions=[g_def]'.
# Not clear if that preserves the original intent. See cell below
# gg_eq = g_def.substitution(gg.inner_expr().operator, [g_def])

In [ ]:
# From hoss: here I changed the original positional arg '[g_def]' to 'assumptions=[g_def]'
# See original cell above
gg_eq = g_def.substitution(gg.inner_expr().operator, assumptions = [g_def])

In [ ]:
gg_impl_gg = Implies(gg, gg).prove()

In [ ]:
gg__impl__gg_impl_A =gg_eq.sub_right_side_into(gg_impl_gg.inner_expr().consequent)

In [ ]:
# From hoss: here I changed the original positional arg '[gg]' to 'assumptions=[gg]'
# Note clear if that preserves the original intent. See cell below
# A_assuming_gg = gg__impl__gg_impl_A.derive_consequent([gg]).derive_consequent()

In [ ]:
# From hoss: here I changed the original positional arg '[gg]' to 'assumptions=[gg]'
# See original cell above
A_assuming_gg = gg__impl__gg_impl_A.derive_consequent(assumptions = [gg]).derive_consequent()

In [ ]:
gg_impl_A = A_assuming_gg.as_implication(gg)

In [ ]:
gg_eq.sub_left_side_into(gg_impl_A)

In [ ]:
gg_impl_A.derive_consequent()

### III. Another approach (20240823)

As WW points out, neither this approach nor the approach above is problematic for the Prove-It system since we are explicitly including the definitional assumption in the derivation(s). This is a kind of “simulation” of the Curry paradox.

In [ ]:
from proveit import A, B, X, Y, Z
from proveit.logic import Boolean, InSet
from proveit.logic.booleans.implication import implication_contraction

In [ ]:
X_def = Equals(X, Implies(X, Y))

In [ ]:
temp_assumptions = [X_def, InSet(X, Boolean), InSet(Y, Boolean)]

In [ ]:
x_implies_x = Implies(X, X).prove(assumptions = temp_assumptions)

In [ ]:
x_implies_x_implies_y = X_def.substitution(
        x_implies_x.inner_expr().rhs, assumptions = temp_assumptions
).derive_right_via_equality()

In [ ]:
implication_contraction

In [ ]:
implication_contraction_inst = implication_contraction.instantiate({A:X, B:Y}, assumptions = temp_assumptions)

In [ ]:
implication_contraction_inst_eq = implication_contraction_inst.derive_equality(assumptions = temp_assumptions)

In [ ]:
x_implies_y = implication_contraction_inst_eq.sub_right_side_into(x_implies_x_implies_y, assumptions = temp_assumptions)

In [ ]:
X_def.sub_left_side_into(x_implies_y, assumptions = temp_assumptions)

In [ ]:
x_implies_y.derive_consequent(assumptions = temp_assumptions)

### III. Continued (20241204)

In [ ]:
from proveit import g, P, Function, Lambda, Operation
from proveit.logic import FALSE, Implies

In [ ]:
g_of_g = Function(g, g)

In [ ]:
the_lambda = Lambda(P, Implies(Function(P, P), FALSE))

In [ ]:
g_of_g.inner_expr().operand

In [ ]:
try:
    g_of_g.inner_expr().operand.instantiate({g: the_lambda})
except Exception as the_exception:
    print(f'Exception: {the_exception}')

### IV. Defining `g` via Explicit Skolemization

Related: see related emails with WW on 8/26 - 8/28/24.

In [ ]:
from proveit import defaults
from proveit import g, P, Function, Lambda
from proveit.logic import Exists, FALSE, Not

In [ ]:
the_lambda = Lambda(P, Implies(Function(P, P), FALSE))

In [ ]:
g_exists = Exists(g, Equals(g, the_lambda), conditions=[]).conclude_via_example(the_lambda)

In [ ]:
g_exists.choose(g)

We can verify that the definition of `g` is now included in the default assumptions:

In [ ]:
defaults.assumptions

In [ ]:
g_def = defaults.assumptions[0]

In [ ]:
gg = Function(g, g)

In [ ]:
gg_eq = g_def.substitution(gg.inner_expr().operator)

In [ ]:
gg_impl_gg = Implies(gg, gg).prove()

In [ ]:
gg__impl__gg_impl_false = gg_eq.sub_right_side_into(gg_impl_gg.inner_expr().consequent)

In [ ]:
false_assuming_gg = gg__impl__gg_impl_false.derive_consequent(assumptions = [gg]).derive_consequent()

In [ ]:
gg_impl_false = false_assuming_gg.as_implication(gg)

In [ ]:
gg_eq.sub_left_side_into(gg_impl_false)

In [ ]:
false_judgment = gg_impl_false.derive_consequent()

In [ ]:
false_judgment.eliminate(g,)

In [ ]:
%end _curry_paradox_hoss